In [ ]:
!pip install pytest aiohttp pytest-aiohttp aiosqlite

In [ ]:
!pip install pytest-sugar

In [ ]:
%%writefile test_01_resolve.py
import pytest
import asyncio

async def async_add(a, b):
    await asyncio.sleep(0.1)
    return a + b

@pytest.mark.asyncio
async def test_async_add_resolves():
    result = await async_add(2, 3)
    assert result == 5

Overwriting test_01_resolve.py


In [ ]:
%%writefile test_02_reject.py
import pytest
import asyncio

async def fails():
    await asyncio.sleep(0.1)
    raise ValueError("wrong value")

@pytest.mark.asyncio
async def test_async_raises():
    with pytest.raises(ValueError, match="wrong value"):
        await fails()

Overwriting test_02_reject.py


In [ ]:
%%writefile test_03_http.py
import pytest
from aiohttp import web

async def handler(request):
    return web.json_response({"status": "ok", "value": 42})

@pytest.fixture
def test_app(aiohttp_client):
    app = web.Application()
    app.router.add_get("/test", handler)
    return aiohttp_client(app)

@pytest.mark.asyncio
async def test_http_request(test_app):
    client = await test_app

    async with client.get("/test") as resp:
        data = await resp.json()

    assert data["status"] == "ok"
    assert data["value"] == 42

Overwriting test_03_http.py


In [ ]:
%%writefile test_04_db.py
import pytest
import aiosqlite

async def insert_user(db, name):
    await db.execute("INSERT INTO users (name) VALUES (?)", (name,))
    await db.commit()

@pytest.mark.asyncio
async def test_db_insert(tmp_path):
    db_path = tmp_path / "test.db"

    async with aiosqlite.connect(db_path) as db:
        await db.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT)")
        await insert_user(db, "Alice")

        cursor = await db.execute("SELECT name FROM users WHERE name='Alice'")
        row = await cursor.fetchone()

    assert row[0] == "Alice"

Overwriting test_04_db.py


In [ ]:
%%writefile test_05_thread.py
import pytest
import asyncio

async def compute():
    await asyncio.sleep(0.1)
    return 10

async def run_in_thread():
    def sync_runner():
        return asyncio.run(compute())
    return await asyncio.to_thread(sync_runner)

@pytest.mark.asyncio
async def test_async_in_thread():
    result = await run_in_thread()
    assert result == 10

Overwriting test_05_thread.py


In [ ]:
!pytest -q

Test session starts (platform: linux, Python 3.12.12, pytest 8.4.2, pytest-sugar 1.1.1)
rootdir: /content
plugins: aiohttp-1.1.0, sugar-1.1.1, asyncio-1.3.0, langsmith-0.4.47, anyio-4.11.0, typeguard-4.4.4
asyncio: mode=Mode.STRICT, debug=False, asyncio_default_fixture_loop_scope=None, asyncio_default_test_loop_scope=function

 test_01_resolve.py ✓                                             20% ██        
 test_02_reject.py ✓                                              40% ████      
 test_03_http.py ✓                                                60% ██████    
 test_04_db.py ✓                                                  80% ████████  
 test_05_thread.py ✓                                             100% ██████████

Results (0.38s):
       5 passed
